In [1]:
import pandas as pd
import numpy as np
import math
import os

# import for time, and file change
import json
from PIL import Image

# frequent updating
import time
import random

In [2]:
# define filepaths
workspace = "/Users/advaysingh/Documents/projects/hindi_classification/" 
#print("Current workspace:", workspace)

data = os.path.join(workspace, 'data/Hindi/')
dict_lib = os.path.join(workspace, 'data/dict.csv')
img_path = os.path.join(workspace, 'server/snapshot.png')
out_file = os.path.join(workspace, 'server/outputs.json')


In [3]:
# create train/test dicts with files and labels

def create_dict(x: str) -> dict:
    x_dict = {}
    i = 0
    for dir in os.listdir(os.path.join(data, x)):
        for file in os.listdir(os.path.join(data, x, dir)):
            x_dict[os.path.join(data, x, dir, file)] = i
        i += 1
    return x_dict

# make pandas df

train_df = pd.DataFrame.from_dict(create_dict('Train'), orient='index')
test_df = pd.DataFrame.from_dict(create_dict('Test'), orient='index')

train_df.head()

,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/26446.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/62562.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/74300.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/86182.png,0
/Users/advaysingh/Documents/projects/hindi_classification/data/Hindi/Train/character_29_waw/62576.png,0


In [4]:
# Activations class
class Activation:
    # can add more types
    def __init__(self, act_type: str) -> None:
        self.act = act_type

    def print_act(self) -> None:
        return self.act
    
    def compute(self, z: float) -> float:

        # Sigmoid function
        if (self.act == 'sigmoid'):
            return 1 / (1 + math.exp(z))
        
        # ReLU function
        elif (self.act == 'relu'):
            if (z < 0): return 0
            return z

    def prime(self, z: float) -> float:

        # Sigmoid prime
        if (self.act == 'sigmoid'):
            return (self.compute(z))(1 - self.compute(z))
        
        # ReLU prime
        if (self.act == 'relu'):
            if (z < 0): return 0
            return 1


def softmax(i: int, costs: list) -> float:
    return costs[i] / sum(costs)

def softmax_prime(i: int, costs: list) -> float:
    softmax(i, costs)(1 - softmax(i, costs))


    
a = Activation('sigmoid')

print(a.compute(233.2))

5.278698047466735e-102


In [69]:
class Model:
    def __init__(self, train_df: pd.DataFrame,
                  act: Activation, layers: list, step: float, stochastic: bool) -> None:
        self.train_df = train_df

        # create activation function
        self.activation = act
        self.layers = layers

        # init weight and bias with # layers np arrays
        self.all_weights = [None] * (int(len(layers)))
        self.all_bias = [None] * (int(len(layers)))
        for i in range(len(layers)):
            self.all_weights[i] = self.random_arrs(i)
            self.all_bias[i] = self.random_arrs(i)

        # init learn speed
        self.speed = step

        # determine gradient decent type
        self.stochastic = stochastic

    def random_arrs(self, layer: int):
        if (layer == 0):
            return(np.random.rand(self.layers[layer], int(len(train_df))))
        else:
            return(np.random.rand(self.layers[layer], self.layers[layer - 1]))

    def img_to_np(self, dir) -> np.array:
        return np.array(Image.open(dir)).flatten()
    
    def print_img(self) -> None:
        for img_dir, row in self.train_df.iterrows():
            print(self.img_to_np(img_dir), row[0])

    """ def forward_prop(self, inputs: list, layer: int, num_outputs: int) -> list: 
        for x in inputs:
            for y in range(num_outputs):"""
    """def forward_prop(self, layer: int, inputs: list) -> list:
        self.weight_arrs[]"""
    def print_weights(self) -> None:
        for arr in self.all_weights:
            print(arr)

    #def back_prop()

    #def_train()

    




In [70]:

cnn = Model(train_df.head(), Activation('sigmoid'), [5, 5, 46], 1.2, False)
cnn.print_img()
cnn.print_weights()

5
5
46
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[0 0 0 ... 0 0 0] 0
[[0.59209418 0.47379698 0.74635361 ... 0.18800198 0.95590137 0.47722192]
 [0.15108503 0.60728361 0.37685904 ... 0.19830556 0.04246842 0.97012312]
 [0.46355416 0.33071766 0.44603848 ... 0.52880412 0.62271191 0.5659487 ]
 [0.26460595 0.33989487 0.05640258 ... 0.77753803 0.30379417 0.49664403]
 [0.92290739 0.79867175 0.16867853 ... 0.73817012 0.02694859 0.83146053]]
[[0.8153661  0.29307366 0.58418489 0.33501517 0.36770677]
 [0.50565913 0.82459524 0.69936184 0.56937528 0.22232651]
 [0.21336175 0.84875973 0.31973573 0.68290335 0.13138255]
 [0.8414451  0.25105682 0.72201321 0.17662217 0.60465154]
 [0.2042809  0.0686943  0.57893146 0.14589292 0.05896684]]
[[0.30572066 0.78952821 0.18964419 0.73840348 0.03901428]
 [0.13142756 0.06281074 0.19179484 0.2882064  0.45287101]
 [0.86144192 0.65092717 0.81696584 0.38842049 0.83028129]
 [0.86880352 0.99552298 0.44285661 0.99255999 0.43526492]
 [0.

In [71]:
import json
import time
import random

from PIL import Image


def update_json_file(hindi_value, english_value):
    with open(out_file, 'r') as file:
        data = json.load(file)
    data['hindi'] = hindi_value
    data['english'] = english_value
    with open(out_file, 'w') as file:
        json.dump(data, file, indent=4)

# Define a list of possible values if applicable
hindi_values = ["हिं", "नया", "मूल्य", "उदाहरण"]
english_values = ["Eng", "New", "Value", "Example"]

# Loop to update the JSON file every second with a random value
try:
    while True:
        # Generate a random value from the lists
        new_hindi_value = random.choice(hindi_values)
        new_english_value = random.choice(english_values)

        update_json_file(new_hindi_value, new_english_value)
        
        # Print the new values (for debug purposes)
        # print(f'Updated JSON file with hindi: {new_hindi_value}, english: {new_english_value}')

        # read in and resize img
        img = Image.open(img_path).convert("L").resize((32, 32))
 
        # read in image for debugging
        """img.show("img to classify")
        print(img.format)
        print(img.mode)
        print(img.size)"""

        # create np array and resize
        img_data = np.array(img).flatten()
        print(img_data.shape)
        print(img_data)

        
        time.sleep(10)  # Wait for 1 second
except KeyboardInterrupt:
    print("Update stopped by user.")

(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[0 0 0 ... 0 0 0]
(1024,)
[67 11 10 ...  0  0  0]
Update stopped by user.
